In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Articles/Satellite_Image_TimeSeriesClassification/

/content/drive/My Drive/Articles/Satellite_Image_TimeSeriesClassification


In [ ]:
pip install torchnet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchnet: filename=torchnet-0.0.4-py3-none-any.whl size=29728 sha256=49ddac07c3c9d684f2638b72ef458652b5a4d9abfe24da5be85006cb9ceae404
  Stored in directory: /root/.cache/pip/wheels/f7/ae/94/9f5edd6871983f30967ad11d60ef434c3d1b007654de4c8065
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408194 sha256=a39d23a5b6b895c552346cd7d752457602a93012a051e0077ea2e0f4fd1bcca4
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built torchnet visdom


In [ ]:
import torch
import torch.utils.data as data
import torchnet as tnt
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import os
import json
import pickle as pkl
import argparse
import pprint

In [ ]:
from models.stclassifier import PseTae
from dataset import PixelSetData, PixelSetData_preloaded
from learning.focal_loss import FocalLoss
from learning.weight_init import weight_init
from learning.metrics import mIou, confusion_matrix_analysis

In [ ]:

def train_epoch(model, optimizer, criterion, data_loader, device, args):
    acc_meter = tnt.meter.ClassErrorMeter(accuracy=True)
    loss_meter = tnt.meter.AverageValueMeter()
    y_true = []
    y_pred = []

    for i, (x, y) in enumerate(data_loader):

        y_true.extend(list(map(int, y)))

        x = recursive_todevice(x, device)
        y = y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y.long())
        loss.backward()
        optimizer.step()

        pred = out.detach()
        y_p = pred.argmax(dim=1).cpu().numpy()
        y_pred.extend(list(y_p))
        acc_meter.add(pred, y)
        loss_meter.add(loss.item())

        if (i + 1) % args['display_step'] == 0:
            print('Step [{}/{}], Loss: {:.4f}, Acc : {:.2f}'.format(i + 1, len(data_loader), loss_meter.value()[0],
                                                                    acc_meter.value()[0]))

    epoch_metrics = {'train_loss': loss_meter.value()[0],
                     'train_accuracy': acc_meter.value()[0],
                     'train_IoU': mIou(y_true, y_pred, n_classes=args['num_classes'])}

    return epoch_metrics


def evaluation(model, criterion, loader, device, args, mode='val'):
    y_true = []
    y_pred = []

    acc_meter = tnt.meter.ClassErrorMeter(accuracy=True)
    loss_meter = tnt.meter.AverageValueMeter()

    for (x, y) in loader:
        y_true.extend(list(map(int, y)))
        x = recursive_todevice(x, device)
        y = y.to(device)

        with torch.no_grad():
            prediction = model(x)
            loss = criterion(prediction, y)

        acc_meter.add(prediction, y)
        loss_meter.add(loss.item())

        y_p = prediction.argmax(dim=1).cpu().numpy()
        y_pred.extend(list(y_p))

    metrics = {'{}_accuracy'.format(mode): acc_meter.value()[0],
               '{}_loss'.format(mode): loss_meter.value()[0],
               '{}_IoU'.format(mode): mIou(y_true, y_pred, args['num_classes'])}

    if mode == 'val':
        return metrics
    elif mode == 'test':
        return metrics, confusion_matrix(y_true, y_pred, labels=list(range(args['num_classes'])))


def get_loaders(dt, kfold, args):
    indices = list(range(len(dt)))
    np.random.shuffle(indices)

    kf = KFold(n_splits=kfold, shuffle=False)
    indices_seq = list(kf.split(list(range(len(dt)))))
    ntest = len(indices_seq[0][1])

    loader_seq = []
    for trainval, test_indices in indices_seq:
        trainval = [indices[i] for i in trainval]
        test_indices = [indices[i] for i in test_indices]

        validation_indices = trainval[-ntest:]
        train_indices = trainval[:-ntest]

        train_sampler = data.sampler.SubsetRandomSampler(train_indices)
        validation_sampler = data.sampler.SubsetRandomSampler(validation_indices)
        test_sampler = data.sampler.SubsetRandomSampler(test_indices)

        train_loader = data.DataLoader(dt, batch_size=args['batch_size'],
                                       sampler=train_sampler,
                                       num_workers=args['num_workers'])
        validation_loader = data.DataLoader(dt, batch_size=args['batch_size'],
                                            sampler=validation_sampler,
                                            num_workers=args['num_workers'])
        test_loader = data.DataLoader(dt, batch_size=args['batch_size'],
                                      sampler=test_sampler,
                                      num_workers=args['num_workers'])

        loader_seq.append((train_loader, validation_loader, test_loader))
    return loader_seq


def recursive_todevice(x, device):
    if isinstance(x, torch.Tensor):
        return x.to(device)
    else:
        return [recursive_todevice(c, device) for c in x]


def prepare_output(args):
    os.makedirs(args['res_dir'], exist_ok=True)
    for fold in range(1, args['kfold'] + 1):
        os.makedirs(os.path.join(args['res_dir'], 'Fold_{}'.format(fold)), exist_ok=True)


def checkpoint(fold, log, args):
    with open(os.path.join(args['res_dir'], 'Fold_{}'.format(fold), 'trainlog.json'), 'w') as outfile:
        json.dump(log, outfile, indent=4)


def save_results(fold, metrics, conf_mat, args):
    with open(os.path.join(args['res_dir'], 'Fold_{}'.format(fold), 'test_metrics.json'), 'w') as outfile:
        json.dump(metrics, outfile, indent=4)
    pkl.dump(conf_mat, open(os.path.join(args['res_dir'], 'Fold_{}'.format(fold), 'conf_mat.pkl'), 'wb'))


def overall_performance(args):
    cm = np.zeros((args['num_classes'], args['num_classes']))
    for fold in range(1, args['kfold'] + 1):
        cm += pkl.load(open(os.path.join(args['res_dir'], 'Fold_{}'.format(fold), 'conf_mat.pkl'), 'rb'))

    _, perf = confusion_matrix_analysis(cm)

    print('Overall performance:')
    print('Acc: {},  IoU: {}'.format(perf['Accuracy'], perf['MACRO_IoU']))

    with open(os.path.join(args['res_dir'], 'overall.json'), 'w') as file:
        file.write(json.dumps(perf, indent=4))


def main(args):
    np.random.seed(args['rdm_seed'])
    torch.manual_seed(args['rdm_seed'])
    prepare_output(args)
    mean_std = pkl.load(open(args['dataset_folder'] + '/S2-2017-T31TFM-meanstd.pkl.pkl', 'rb'))


    #mean_std = pkl.load(open('/content/drive/MyDrive/Articles/Satellite_Image_TimeSeriesClassification/dataset_folder/S2-2017-T31TFM-meanstd.pkl', 'rb'))




    extra = 'geomfeat' if args['geomfeat'] else None

    if args['preload']:
        dt = PixelSetData_preloaded(args['dataset_folder'], labels='label_44class', npixel=args['npixel'],
                          sub_classes=[1, 3, 4, 5, 6, 8, 9, 12, 13, 14, 16, 18, 19, 23, 28, 31, 33, 34, 36, 39],
                          norm=mean_std,
                          extra_feature=extra)
    else:
        dt = PixelSetData(args['dataset_folder'], labels='label_44class', npixel=args['npixel'],
                          sub_classes=[1, 3, 4, 5, 6, 8, 9, 12, 13, 14, 16, 18, 19, 23, 28, 31, 33, 34, 36, 39],
                          norm=mean_std,
                          extra_feature=extra)
    device = torch.device(args['device'])

    loaders = get_loaders(dt, args['kfold'], args)
    for fold, (train_loader, val_loader, test_loader) in enumerate(loaders):
        print('Starting Fold {}'.format(fold + 1))
        print('Train {}, Val {}, Test {}'.format(len(train_loader), len(val_loader), len(test_loader)))

        model_args= dict(input_dim=args['input_dim'], mlp1=args['mlp1'], pooling=args['pooling'],
                            mlp2=args['mlp2'], n_head=args['n_head'], d_k=args['d_k'], mlp3=args['mlp3'],
                            dropout=args['dropout'], T=args['T'], len_max_seq=args['lms'],
                            positions=dt.date_positions if args['positions'] == 'bespoke' else None,
                            mlp4=args['mlp4'])

        if args['geomfeat']:
            model_args.update(with_extra=True, extra_size=4)
        else:
            model_args.update(with_extra=False, extra_size=None)

        model = PseTae(**model_args)

        print(model.param_ratio())

        model = model.to(device)
        model.apply(weight_init)
        optimizer = torch.optim.Adam(model.parameters())
        criterion = FocalLoss(args['gamma'])

        trainlog = {}



        best_mIoU = 0
        for epoch in range(1, args['epochs'] + 1):
            print('EPOCH {}/{}'.format(epoch, args['epochs']))

            model.train()
            train_metrics = train_epoch(model, optimizer, criterion, train_loader, device=device, args=args)

            print('Validation . . . ')
            model.eval()
            val_metrics = evaluation(model, criterion, val_loader, device=device, args=args, mode='val')

            print('Loss {:.4f},  Acc {:.2f},  IoU {:.4f}'.format(val_metrics['val_loss'], val_metrics['val_accuracy'],
                                                                 val_metrics['val_IoU']))

            trainlog[epoch] = {**train_metrics, **val_metrics}
            checkpoint(fold + 1, trainlog, args)

            if val_metrics['val_IoU'] >= best_mIoU:
                best_mIoU = val_metrics['val_IoU']
                torch.save({'epoch': epoch, 'state_dict': model.state_dict(),
                            'optimizer': optimizer.state_dict()},
                           os.path.join(args['res_dir'], 'Fold_{}'.format(fold + 1), 'model.pth.tar'))

        print('Testing best epoch . . .')
        model.load_state_dict(
            torch.load(os.path.join(args['res_dir'], 'Fold_{}'.format(fold + 1), 'model.pth.tar'))['state_dict'])
        model.eval()

        test_metrics, conf_mat = evaluation(model, criterion, test_loader, device=device, mode='test', args=args)

        print('Loss {:.4f},  Acc {:.2f},  IoU {:.4f}'.format(test_metrics['test_loss'], test_metrics['test_accuracy'],
                                                             test_metrics['test_IoU']))
        save_results(fold + 1, test_metrics, conf_mat, args)

    overall_performance(args)



In [ ]:

if __name__ == '__main__':

    parser = argparse.ArgumentParser()

    # Set-up parameters
    parser.add_argument('--dataset_folder', default='/content/drive/MyDrive/Articles/Satellite_Image_TimeSeriesClassification/dataset_folder', type=str,
                        help='Path to the folder where the results are saved.')
    parser.add_argument('--res_dir', default='./results', help='Path to the folder where the results should be stored')
    parser.add_argument('--num_workers', default=8, type=int, help='Number of data loading workers')
    parser.add_argument('--rdm_seed', default=1, type=int, help='Random seed')
    parser.add_argument('--device', default='cuda', type=str,
                        help='Name of device to use for tensor computations (cuda/cpu)')
    parser.add_argument('--display_step', default=50, type=int,
                        help='Interval in batches between display of training metrics')
    parser.add_argument('--preload', dest='preload', action='store_true',
                        help='If specified, the whole dataset is loaded to RAM at initialization')
    parser.set_defaults(preload=False)

    # Training parameters
    parser.add_argument('--kfold', default=5, type=int, help='Number of folds for cross validation')
    parser.add_argument('--epochs', default=100, type=int, help='Number of epochs per fold')
    parser.add_argument('--batch_size', default=128, type=int, help='Batch size')
    parser.add_argument('--lr', default=0.001, type=float, help='Learning rate')
    parser.add_argument('--gamma', default=1, type=float, help='Gamma parameter of the focal loss')
    parser.add_argument('--npixel', default=64, type=int, help='Number of pixels to sample from the input images')

    # Architecture Hyperparameters
    ## PSE
    parser.add_argument('--input_dim', default=10, type=int, help='Number of channels of input images')
    parser.add_argument('--mlp1', default='[10,32,64]', type=str, help='Number of neurons in the layers of MLP1')
    parser.add_argument('--pooling', default='mean_std', type=str, help='Pixel-embeddings pooling strategy')
    parser.add_argument('--mlp2', default='[132,128]', type=str, help='Number of neurons in the layers of MLP2')
    parser.add_argument('--geomfeat', default=1, type=int,
                        help='If 1 the precomputed geometrical features (f) are used in the PSE.')

    ## TAE
    parser.add_argument('--n_head', default=4, type=int, help='Number of attention heads')
    parser.add_argument('--d_k', default=32, type=int, help='Dimension of the key and query vectors')
    parser.add_argument('--mlp3', default='[512,128,128]', type=str, help='Number of neurons in the layers of MLP3')
    parser.add_argument('--T', default=1000, type=int, help='Maximum period for the positional encoding')
    parser.add_argument('--positions', default='bespoke', type=str,
                        help='Positions to use for the positional encoding (bespoke / order)')
    parser.add_argument('--lms', default=None, type=int,
                        help='Maximum sequence length for positional encoding (only necessary if positions == order)')
    parser.add_argument('--dropout', default=0.2, type=float, help='Dropout probability')

    ## Classifier
    parser.add_argument('--num_classes', default=20, type=int, help='Number of classes')
    parser.add_argument('--mlp4', default='[128, 64, 32, 20]', type=str, help='Number of neurons in the layers of MLP4')



In [9]:
    args= parser.parse_args(args=[])
    args= vars(args)
    for k, v in args.items():
        if 'mlp' in k:
            v = v.replace('[', '')
            v = v.replace(']', '')
            args[k] = list(map(int, v.split(',')))

    pprint.pprint(args)
    main(args)

{'T': 1000,
 'batch_size': 128,
 'd_k': 32,
 'dataset_folder': '/content/drive/MyDrive/Articles/Satellite_Image_TimeSeriesClassification/dataset_folder',
 'device': 'cuda',
 'display_step': 50,
 'dropout': 0.2,
 'epochs': 100,
 'gamma': 1,
 'geomfeat': 1,
 'input_dim': 10,
 'kfold': 5,
 'lms': None,
 'lr': 0.001,
 'mlp1': [10, 32, 64],
 'mlp2': [132, 128],
 'mlp3': [512, 128, 128],
 'mlp4': [128, 64, 32, 20],
 'n_head': 4,
 'npixel': 64,
 'num_classes': 20,
 'num_workers': 8,
 'pooling': 'mean_std',
 'positions': 'bespoke',
 'preload': False,
 'rdm_seed': 1,
 'res_dir': './results'}


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Starting Fold 1
Train 5, Val 2, Test 2
TOTAL TRAINABLE PARAMETERS : 164116
RATIOS: Spatial  12.1% , Temporal  81.0% , Classifier   6.8%
None
EPOCH 1/100


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validation . . . 
Loss 4.9130,  Acc 0.00,  IoU 0.0000
EPOCH 2/100
Validation . . . 
Loss 4.9013,  Acc 0.00,  IoU 0.0000
EPOCH 3/100
Validation . . . 
Loss 4.5986,  Acc 0.00,  IoU 0.0000
EPOCH 4/100
Validation . . . 
Loss 4.0633,  Acc 0.00,  IoU 0.0000
EPOCH 5/100
Validation . . . 
Loss 3.1234,  Acc 6.50,  IoU 0.0081
EPOCH 6/100
Validation . . . 
Loss 2.2466,  Acc 25.00,  IoU 0.0409
EPOCH 7/100
Validation . . . 
Loss 1.6827,  Acc 46.50,  IoU 0.0882
EPOCH 8/100
Validation . . . 
Loss 1.2236,  Acc 72.50,  IoU 0.2019
EPOCH 9/100
Validation . . . 
Loss 1.0297,  Acc 83.00,  IoU 0.2693
EPOCH 10/100
Validation . . . 
Loss 0.9024,  Acc 86.50,  IoU 0.3280
EPOCH 11/100
Validation . . . 
Loss 0.8041,  Acc 89.00,  IoU 0.3467
EPOCH 12/100
Validation . . . 
Loss 0.7224,  Acc 90.50,  IoU 0.3402
EPOCH 13/100
Validation . . . 
Loss 0.6557,  Acc 91.50,  IoU 0.3321
EPOCH 14/100
Validation . . . 
Loss 0.5915,  Acc 91.50,  IoU 0.3420
EPOCH 15/100
Validation . . . 
Loss 0.5429,  Acc 92.00,  IoU 0.3205
EPOCH 

...

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validation . . . 
Loss 2.8503,  Acc 0.50,  IoU 0.0006
EPOCH 2/100
Validation . . . 
Loss 3.0409,  Acc 18.00,  IoU 0.0608
EPOCH 3/100
Validation . . . 
Loss 3.3862,  Acc 23.00,  IoU 0.0483
EPOCH 4/100
Validation . . . 
Loss 3.2815,  Acc 24.50,  IoU 0.0490
EPOCH 5/100
Validation . . . 
Loss 2.8509,  Acc 25.00,  IoU 0.0510
EPOCH 6/100
Validation . . . 
Loss 2.2799,  Acc 25.00,  IoU 0.0611
EPOCH 7/100
Validation . . . 
Loss 1.8666,  Acc 26.50,  IoU 0.0720
EPOCH 8/100
Validation . . . 
Loss 1.5730,  Acc 41.00,  IoU 0.1731
EPOCH 9/100
Validation . . . 
Loss 1.3646,  Acc 63.50,  IoU 0.2282
EPOCH 10/100
Validation . . . 
Loss 1.1475,  Acc 77.50,  IoU 0.2500
EPOCH 11/100
Validation . . . 
Loss 0.9828,  Acc 84.50,  IoU 0.2631
EPOCH 12/100
Validation . . . 
Loss 0.8641,  Acc 88.50,  IoU 0.2732
EPOCH 13/100
Validation . . . 
Loss 0.7579,  Acc 90.00,  IoU 0.3508
EPOCH 14/100
Validation . . . 
Loss 0.7068,  Acc 91.00,  IoU 0.3492
EPOCH 15/100
Validation . . . 
Loss 0.6525,  Acc 91.50,  IoU 0.3563
EP

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validation . . . 
Loss 3.2825,  Acc 0.00,  IoU 0.0000
EPOCH 2/100
Validation . . . 
Loss 3.2023,  Acc 0.00,  IoU 0.0000
EPOCH 3/100
Validation . . . 
Loss 3.2609,  Acc 0.00,  IoU 0.0000
EPOCH 4/100
Validation . . . 
Loss 2.9519,  Acc 1.50,  IoU 0.0044
EPOCH 5/100
Validation . . . 
Loss 2.3348,  Acc 15.50,  IoU 0.0453
EPOCH 6/100
Validation . . . 
Loss 1.7985,  Acc 32.50,  IoU 0.0950
EPOCH 7/100
Validation . . . 
Loss 1.5692,  Acc 38.50,  IoU 0.0965
EPOCH 8/100
Validation . . . 
Loss 1.4319,  Acc 49.50,  IoU 0.1611
EPOCH 9/100
Validation . . . 
Loss 1.3001,  Acc 65.00,  IoU 0.2459
EPOCH 10/100
Validation . . . 
Loss 1.1545,  Acc 72.00,  IoU 0.2819
EPOCH 11/100
Validation . . . 
Loss 1.0349,  Acc 74.00,  IoU 0.2783
EPOCH 12/100
Validation . . . 
Loss 0.9620,  Acc 76.00,  IoU 0.2844
EPOCH 13/100
Validation . . . 
Loss 0.7894,  Acc 81.00,  IoU 0.3059
EPOCH 14/100
Validation . . . 
Loss 0.6949,  Acc 83.50,  IoU 0.3295
EPOCH 15/100
Validation . . . 
Loss 0.7050,  Acc 84.50,  IoU 0.3314
EPOCH

/content/drive/MyDrive/Articles/Satellite_Image_TimeSeriesClassification/learning/metrics.py:61: RuntimeWarning: invalid value encountered in double_scalars
  d['Precision'] = tp / (tp + fp)
/content/drive/MyDrive/Articles/Satellite_Image_TimeSeriesClassification/learning/metrics.py:60: RuntimeWarning: invalid value encountered in double_scalars
  d['IoU'] = tp / (tp + fp + fn)
/content/drive/MyDrive/Articles/Satellite_Image_TimeSeriesClassification/learning/metrics.py:62: RuntimeWarning: invalid value encountered in double_scalars
  d['Recall'] = tp / (tp + fn)
/content/drive/MyDrive/Articles/Satellite_Image_TimeSeriesClassification/learning/metrics.py:63: RuntimeWarning: invalid value encountered in double_scalars
  d['F1-score'] = 2 * tp / (2 * tp + fp + fn)
